In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setAppName('StudentData')
sc = SparkContext.getOrCreate(conf=conf)

In [10]:
studentRDD = sc.textFile('data/Student_Data.csv')
header = studentRDD.first()
studentRDD = studentRDD.filter(lambda x: x != header) #Removing the header

#### No. of Students

In [25]:
totalStudents = studentRDD.map(lambda x: x.split(',')[2]).distinct().count()
print(f'Total Students in the class are {totalStudents}')

Total Students in the class are 50


#### Total Marks by Males and Females

In [29]:
studentRDD.map(lambda x: x.split(',')).map(lambda x: (x[1], int(x[5]))).reduceByKey(lambda x,y: x+y).collect()

[('Female', 29636), ('Male', 30461)]

#### Pass and Fail Count

In [38]:
marksRDD = studentRDD.map(lambda x: x.split(',')).map(lambda x: (x[2], int(x[5]))).map(lambda x: ('Pass' if x[1] > 50 else 'Fail',x[1]))
marksRDD.countByKey()

defaultdict(int, {'Fail': 370, 'Pass': 630})

In [43]:
marksRDD.map(lambda x: (x[0],1)).reduceByKey(lambda x,y: x+y).collect()

[('Pass', 630), ('Fail', 370)]

#### No. of Students enrolled per Course

In [47]:
studentRDD.map(lambda x: x.split(',')).map(lambda x: (x[3],x[2])).countByKey()

defaultdict(int,
            {'Cloud': 192,
             'DB': 157,
             'DSA': 176,
             'MVC': 157,
             'OOP': 152,
             'PF': 166})

#### Total Marks Per Course

In [49]:
studentRDD.map(lambda x: x.split(',')).map(lambda x: (x[3],int(x[5]))).reduceByKey(lambda x,y: x+y).collect()

[('DB', 9270),
 ('Cloud', 11443),
 ('PF', 9933),
 ('MVC', 9585),
 ('OOP', 8916),
 ('DSA', 10950)]

#### Average Marks per course

In [55]:
stageRDD = studentRDD.map(lambda x: x.split(',')).map(lambda x: (x[3],(int(x[5]),1))).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
stageRDD.map(lambda x: (x[0], x[1][0]/x[1][1])).collect()

[('DB', 59.044585987261144),
 ('Cloud', 59.598958333333336),
 ('PF', 59.83734939759036),
 ('MVC', 61.05095541401274),
 ('OOP', 58.6578947368421),
 ('DSA', 62.21590909090909)]

#### Min Max Marks Per Course

In [61]:
studentRDD.map(lambda x: x.split(',')).map(lambda x: (x[3],int(x[5]))).groupByKey().map(lambda x: (x[0], min(x[1]), max(x[1]))).collect()

[('DB', 20, 98),
 ('Cloud', 20, 99),
 ('PF', 20, 99),
 ('MVC', 22, 99),
 ('OOP', 20, 99),
 ('DSA', 20, 99)]

#### Average Age of Male and Females

In [62]:
stageRDD = studentRDD.map(lambda x: x.split(',')).map(lambda x: (x[1],(int(x[0]),1))).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
stageRDD.map(lambda x: (x[0], x[1][0]/x[1][1])).collect()

[('Female', 28.489021956087825), ('Male', 28.52304609218437)]

In [63]:
sc.stop()